<a href="https://colab.research.google.com/github/UcefMountacer/Classification-Tennis-Winner-prediction/blob/master/keras_on_signal_features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
import keras
import numpy as np
import pandas as pd
from sklearn import preprocessing
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
import math
from keras import layers

In [74]:
data = pd.read_csv('data.csv')
#data = dataf.drop(['Unnamed: 0'], axis = 1)
#data.head()
data.head()

,max_stft,min_stft,std_stft,amp_stft,mean_stft,max_cqt,min_cqt,std_cqt,amp_cqt,mean_cqt,max_cens,min_cens,std_cens,amp_cens,mean_cens,max_mfcc,min_mfcc,std_mfcc,amp_mfcc,mean_mfcc,max_rms,min_rms,std_rms,amp_rms,mean_rms,max_centroid,min_centroid,std_centroid,amp_centroid,mean_centroid,max_bandwidth,min_bandwidth,std_bandwidth,amp_bandwidth,mean_bandwidth,max_contrast,min_contrast,std_contrast,amp_contrast,mean_contrast,max_rolloff,min_rolloff,std_rolloff,amp_rolloff,mean_rolloff,max_poly,min_poly,std_poly,amp_poly,mean_poly,max_tonnetz,min_tonnetz,std_tonnetz,amp_tonnetz,mean_tonnetz,max_z_crossing,min_z_crossing,std_z_crossing,amp_z_crossing,mean_z_crossing,label
mot,1.0,0.003162,0.316297,0.996838,0.441173,1.0,0.056944,0.248739,0.943056,0.596353,0.434786,0.132916,0.066434,0.301870,0.280927,164.61353,-445.97928,83.208030,610.5928,-24.938541,7.275048,0.036954,2.054749,7.238094,1.174051,2843.710775,995.874470,678.671596,1847.836305,2099.727158,1920.380683,752.530596,401.224798,1167.850087,1426.284066,63.557396,6.214472,15.714731,57.342924,24.062522,850.561523,312.231445,129.995471,538.330078,553.513747,850.561523,312.231445,129.995471,538.330078,553.513747,9.869232,-0.002795,1.783867,9.872026,0.565800,0.171421,-0.282936,0.056893,0.454357,-0.008081,1
mot,1.0,0.078545,0.268094,0.921455,0.463307,1.0,0.179530,0.202865,0.820470,0.636913,0.477447,0.166989,0.065695,0.310458,0.281101,201.04535,-484.13147,100.218605,685.1768,-13.066352,0.806889,0.165400,0.178395,0.641489,0.373742,4370.148758,957.790357,946.850944,3412.358401,1917.576049,2634.438485,1169.696540,333.201829,1464.741945,1857.359999,79.422157,5.696201,17.736284,73.725956,23.544791,1894.921875,0.000000,427.840530,1894.921875,161.009632,1894.921875,0.000000,427.840530,1894.921875,161.009632,1.311382,-0.000381,0.259495,1.311763,0.119804,0.278661,-0.110654,0.052441,0.389315,0.016491,1
mot,1.0,0.040983,0.255210,0.959017,0.424412,1.0,0.098441,0.218362,0.901559,0.641359,0.434956,0.113830,0.066238,0.321126,0.280973,199.07516,-475.45294,101.326270,674.5281,-12.804263,1.474483,0.153053,0.309796,1.321430,0.424523,2060.507581,796.648223,279.335686,1263.859358,1309.581381,2295.395625,1124.090005,292.289494,1171.305620,1686.311235,68.431743,5.917790,16.625143,62.513953,23.127938,462.963867,0.000000,137.201527,462.963867,66.067782,462.963867,0.000000,137.201527,462.963867,66.067782,2.375809,-0.000668,0.415179,2.376477,0.171203,0.195716,-0.172491,0.046696,0.368207,0.010726,1
mot,1.0,0.001296,0.314311,0.998704,0.358426,1.0,0.023845,0.262704,0.976155,0.544668,0.453929,0.096239,0.092101,0.357690,0.273589,245.44238,-549.84644,102.009270,795.2888,-20.024490,8.008332,0.005332,2.453288,8.003000,1.379372,3339.308272,598.354310,573.943155,2740.953962,1717.282530,2086.928431,443.769745,463.122601,1643.158686,1565.233713,66.613121,5.240172,13.493151,61.372949,23.277860,613.696289,118.432617,155.686917,495.263672,288.006592,613.696289,118.432617,155.686917,495.263672,288.006592,9.620457,-0.002903,2.025718,9.623360,0.609253,0.213917,-0.361766,0.068973,0.575682,-0.006515,1
mot,1.0,0.003175,0.309502,0.996825,0.455292,1.0,0.052506,0.286496,0.947494,0.535967,0.537587,0.071875,0.098331,0.465711,0.271412,252.81818,-538.63904,101.869580,791.4572,-17.787687,8.482393,0.006984,2.548095,8.475409,1.693850,2372.740133,425.389241,723.015094,1947.350892,1578.383070,1977.283459,390.646502,593.882655,1586.636957,1426.533319,59.790315,5.233026,13.460595,54.557289,23.049548,430.664062,107.666016,95.681882,322.998047,231.481934,430.664062,107.666016,95.681882,322.998047,231.481934,9.205145,-0.002993,2.127365,9.208139,0.723904,0.232015,-0.340769,0.078121,0.572784,-0.009237,1


In [76]:
columns = ['max_stft','min_stft','std_stft','amp_stft','mean_stft',
              'max_cqt','min_cqt','std_cqt','amp_cqt','mean_cqt',
              'max_cens','min_cens','std_cens','amp_cens','mean_cens',
              'max_mfcc','min_mfcc','std_mfcc','amp_mfcc','mean_mfcc',
              'max_rms','min_rms','std_rms','amp_rms','mean_rms',
              'max_centroid','min_centroid','std_centroid','amp_centroid','mean_centroid',
              'max_bandwidth','min_bandwidth','std_bandwidth','amp_bandwidth','mean_bandwidth',
              'max_contrast','min_contrast','std_contrast','amp_contrast','mean_contrast',
              'max_rolloff','min_rolloff','std_rolloff','amp_rolloff','mean_rolloff',
              'max_poly','min_poly','std_poly','amp_poly','mean_poly',
              'max_tonnetz','min_tonnetz','std_tonnetz','amp_tonnetz','mean_tonnetz',
              'max_z_crossing','min_z_crossing','std_z_crossing','amp_z_crossing','mean_z_crossing']
Xx=data[columns].values
yx=data['label'].values
X=Xx[0:4720,:]
y=yx[0:4720]
X=preprocessing.StandardScaler().fit(X).transform(X.astype(float))
print(X.shape, y.shape)

(4720, 60) (4720,)


In [0]:
# one hot encoding
encoder = LabelEncoder()
encoder.fit(y)
encoded_y = encoder.transform(y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_y)


In [0]:
# define baseline model
def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(100, activation='relu', input_dim=60))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(30, activation='relu'))
    model.add(Dense(2, activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
    return model

In [0]:
'''
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(32, (3, 3), input_shape=(54, 84, 4),activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.Conv2D(32, (3, 3),activation=tf.nn.relu))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.Conv2D(32, (3, 3),activation=tf.nn.relu))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.Conv2D(32, (3, 3),activation=tf.nn.relu))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(128, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(128, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(30, activation=tf.nn.softmax))
'''

In [83]:
estimator = KerasClassifier(build_fn=baseline_model, epochs=50, batch_size=100, verbose=0)
estimator

In [0]:
kfold = KFold(n_splits=10, shuffle=True, random_state=3)

In [87]:
cross_val_score(estimator, X, dummy_y, cv=10)

array([0.75423731, 0.72669493, 0.81144068, 0.75635593, 0.74364408,
       0.77542374, 0.79237288, 0.72669492, 0.78177965, 0.66949152])